In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

sns.set_style("whitegrid", {'axes.grid' : False})

from tqdm.auto import tqdm

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import cv2
import imutils
import pickle

In [4]:
from utils.utils import open_video, crop_margins, map_frames, get_game_margins

In [ ]:
screen_to_frame = pickle.load(open("data/screen_to_frame.p", "rb"))
# screen_to_frame = np.array([v.ravel() for v in screen_to_frame.values()])

In [ ]:
# check if all screens are present

fig, axs = plt.subplots(figsize=(16, 96))

for i, s in tqdm(enumerate(screen_to_frame.values())):
    plt.title(f"Screen {i}")
    plt.subplot(22, 2, i + 1)
    plt.imshow(s)

fig.tight_layout()

what am i going to present:

width 60 of every video

last screen corner case
detect not continuous video corner

find global position of game on screen
* calculate the number of unique pixels from the middle
* or KISS, detect the black line on top

time spent on each screen
* detect the screen using nearest neighbor
* get dictionary `{screen: [frames]}`
* for each screen calculate the proportion to the total time since start speedrun

heatmap of each screen
* detect the screen
* detect the kings location (relative to screen and absolute coordinates)
    * one shot object detection
    * opencv template matching
    * moving object detection (if does not move => previous location)
* produce heatmap

pipeline:
* map_screens.py and screen_to_frame.py for static files
* map_frames(filename, screen_to_frame)
* for each screen in screen to frames detect motion

https://www.sicara.ai/blog/en/object-detection-template-matching

https://debuggercafe.com/moving-object-detection-using-frame-differencing-with-opencv/

https://towardsdatascience.com/build-a-motion-heatmap-videousing-opencv-with-python-fd806e8a2340

https://github.com/kjw0612/awesome-deep-vision#imagenet-classification

-

https://www.pyimagesearch.com/2015/05/25/basic-motion-detection-and-tracking-with-python-and-opencv/

In [5]:
screen_to_frame = pickle.load(open("data/screen_to_frame.p", "rb"))
screen_to_frame = np.array([v.ravel() for v in screen_to_frame.values()])

In [6]:
screen_to_frames, frame_to_screen = map_frames('data/speedrun_side.mp4', screen_to_frame)

getting video start


  0%|          | 0/11277 [00:00<?, ?it/s]

video starts at frame 325
mapping screens


  0%|          | 0/10952 [00:00<?, ?it/s]

done mapping screens


In [142]:
cap = open_video('data/speedrun_side.mp4')
margin_left, margin_right = get_game_margins(cap)
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

True

In [143]:
s = screen_to_frames[5][0]
n = s[1] - s[0]
cap.set(cv2.CAP_PROP_POS_FRAMES, s[0])

True

In [144]:
frames = []

for i in tqdm(range(n-1)):
    ret, frame = cap.read()
    if ret == False:
        print("error")
        break
    
    frame = crop_margins(frame, margin_left, margin_right)
    frame = imutils.resize(frame, width=60)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frames.append(frame)

  0%|          | 0/219 [00:00<?, ?it/s]

In [145]:
avg_frame = np.mean(frames, axis=0).astype(np.uint8)

In [148]:
cap.set(cv2.CAP_PROP_POS_FRAMES, s[0])

True

find closest to previous position
make array of starting positions

remember that x y is reversed for frame

In [150]:
frame.shape # 

(44, 60)

In [149]:
def dist(a, b):
    return (a[0]-b[0])**2 + (a[1]-b[1])**2

positions = [(14, 30)]
for i in tqdm(range(n-1)):
    ret, frame = cap.read()
    if ret == False:
        print("error")
        break
    
    frame = crop_margins(frame, margin_left, margin_right)
    frame = imutils.resize(frame, width=60)
    orig_frame = frame.copy()
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_diff = cv2.absdiff(avg_frame, frame)
    
    ret, thres = cv2.threshold(frame_diff, 50, 255, cv2.THRESH_BINARY)
    dilate_frame = cv2.dilate(thres, None, iterations=2)
    
    contours, hierarchy = cv2.findContours(dilate_frame, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # draw the contours, not strictly necessary
    for i, cnt in enumerate(contours):
        cv2.drawContours(frame, contours, i, (0, 0, 255), 3)
    
    if len(contours) == 0:
        positions.append(positions[-1])
    elif len(contours) == 1:
        (x, y, w, h) = cv2.boundingRect(contours[0])
        if dist(positions[-1], (x+w//2, y+h//2)) > 100:
            # detected another object
            positions.append(positions[-1])
        else:
            positions.append((x+w//2, y+h//2))
    else:
        
        distances = []
        for contour in contours:
            (x, y, w, h) = cv2.boundingRect(contour)
            distances.append(dist(positions[-1], (x+w//2, y+h//2)))
        pidx = np.argmin(distances)
        (x, y, w, h) = cv2.boundingRect(contours[pidx])
        positions.append((x+w//2, y+h//2))

    x, y = positions[-1]
    cv2.rectangle(orig_frame, (x-3, y-3), (x+3, y+3), (0, 255, 0), 1)
    
    orig_frame = imutils.resize(orig_frame, width=600)
    cv2.imshow("view", orig_frame)
    k = cv2.waitKey(100) & 0xFF
    if k == ord('q'):
        break
        

cv2.destroyAllWindows()

  0%|          | 0/219 [00:00<?, ?it/s]

In [138]:
cap.release()

detect something like a box

if cannot => previous position

corner case first stage

In [ ]:
king = cv2.imread('data/king.webp')

In [ ]:
plt.imshow(king)

In [ ]:
king.shape